In [1]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight

Using TensorFlow backend.


In [2]:
gc.enable()

In [3]:
steps = 5
height = 108
width = 108

In [4]:
#change the path to where you save the input numpy arrays
lstm = np.array(glob.glob('E:/notes_database/lstm_inputs/*'))
cnn = np.array(glob.glob('E:/notes_database/cnn_inputs/*'))
y_train = np.array(glob.glob('E:/notes_database/y_train/*'))

In [5]:
#The function analyzes the actual accuracy
def load_test_score(model, cnn_test, lstm_test, y_test):
    prediction = model.predict([cnn_test, lstm_test])
    error = (prediction - y_test) ** 2
    error = (error/y_test.shape[0]).sum()
    wrong = 0
    for x in range(prediction.shape[0]):
        for y in range(prediction.shape[1]):
            if abs(prediction[x][y] - y_test[x][y]) > 0.5:
                wrong += 1
                break
    accuracy = 1 - wrong/(prediction.shape[0])
    return error, accuracy

In [6]:
def inputs_to_model(order, num, start, ending):
    cnn_inputs = np.load(cnn[order[num][0]])
    labels_inputs = np.load(y_train[order[num][0]])
    lstm_inputs = np.load(lstm[order[num][0]])
    
    count = 0
    ce = 0
    for file in order[num][start+1:ending]:
        if count == ce + 50:
            print(count)
            ce = count
        count+=1
        
        lstm_inputs = np.append(lstm_inputs, np.load(lstm[file]), axis=0)
        cnn_inputs = np.append(cnn_inputs, np.load(cnn[file]), axis=0)
        labels_inputs = np.append(labels_inputs, np.load(y_train[file]), axis=0)
        
    return lstm_inputs, cnn_inputs, labels_inputs

In [7]:
def main(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [8]:
def main_2(optimizer, loss):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = Dropout(0.3)(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = Flatten()(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32, return_sequences=True)(lstm_layers)
    lstm_layers = LSTM(64, return_sequences=True)(lstm_layers)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

In [9]:
def main_3(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [17]:
def train_test_whole(classifier, num, name, epochs=100, batchsize=32):
    lstm_inputs, cnn_inputs, labels_inputs = inputs_to_model(train_order, num, 0, train_order[num].shape[0]+1)
    
    classifier.fit([cnn_inputs, lstm_inputs], labels_inputs, epochs=epochs, batch_size=batchsize, verbose=1,
                  validation_data = ([cnn_test, lstm_test], labels_test))
    
    #CV_loss, accuracy = load_test_score(classifier, cnn_test, lstm_test, y_test)
    #print('val_loss: ', CV_loss)
    #print('val_accuarcy', accuracy)
    
    classifier.save_weights(name, overwrite=True)   
    del classifier
    K.clear_session()

In [14]:
#Generate the CV sets
kf = KFold(n_splits=3, random_state=7)

train_order = []
test_order = []
length = np.arange(len(lstm))

for x,y in kf.split(length):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [15]:
lstm_test, cnn_test, labels_test = inputs_to_model(test_order, 0, 0, test_order[0].shape[0]+1)

50


In [12]:
adam = keras.optimizers.Adam(lr=0.01)
classifier = main_3(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-spectrums2-simpler-lstmcnn.hdf5', epochs=50)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':


50
50
100
Train on 72054 samples, validate on 36454 samples
Epoch 1/50
72054/72054 [==============================] - 74s 1ms/step - loss: 0.0590 - acc: 0.9805 - val_loss: 0.1714 - val_acc: 0.9374
Epoch 2/50
72054/72054 [==============================] - 69s 951us/step - loss: 0.0344 - acc: 0.9895 - val_loss: 0.0422 - val_acc: 0.9886
Epoch 3/50
72054/72054 [==============================] - 69s 964us/step - loss: 0.0304 - acc: 0.9909 - val_loss: 0.0405 - val_acc: 0.9891
Epoch 4/50
72054/72054 [==============================] - 68s 949us/step - loss: 0.0282 - acc: 0.9915 - val_loss: 0.0288 - val_acc: 0.9917
Epoch 5/50
72054/72054 [==============================] - 69s 960us/step - loss: 0.0267 - acc: 0.9920 - val_loss: 0.0259 - val_acc: 0.9924
Epoch 6/50
72054/72054 [==============================] - 70s 965us/step - loss: 0.0256 - acc: 0.9923 - val_loss: 0.0294 - val_acc: 0.9917
Epoch 7/50
72054/72054 [==============================] - 71s 981us/step - loss: 0.0244 - acc: 0.9927 - val_

18648

In [18]:
classifier = main_3(loss='binary_crossentropy')
classifier.load_weights('weight-spectrums2-simpler-lstmcnn.hdf5')
load_test_score(classifier, cnn_test, lstm_test, labels_test)

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':


(0.13227028, 0.9193229823887639)

In [14]:
adam = keras.optimizers.Adam(lr=0.01)
classifier = main_2(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-spectrums2-main_2-lr0.01.hdf5', epochs=30)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  import sys
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  # This is added back by InteractiveShellApp.init_path()
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


50
50
100
Train on 72054 samples, validate on 36454 samples
Epoch 1/30
72054/72054 [==============================] - 120s 2ms/step - loss: 0.0976 - acc: 0.9665 - val_loss: 0.0847 - val_acc: 0.9715
Epoch 2/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0510 - acc: 0.9844 - val_loss: 0.1273 - val_acc: 0.9759
Epoch 3/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0431 - acc: 0.9871 - val_loss: 0.1006 - val_acc: 0.9826
Epoch 4/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0384 - acc: 0.9886 - val_loss: 0.0864 - val_acc: 0.9866
Epoch 5/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0353 - acc: 0.9896 - val_loss: 0.1546 - val_acc: 0.9788
Epoch 6/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0332 - acc: 0.9903 - val_loss: 0.1435 - val_acc: 0.9763
Epoch 7/30
72054/72054 [==============================] - 116s 2ms/step - loss: 0.0315 - acc: 0.9908 - val_loss:

64158

In [43]:
adam = keras.optimizers.Adam(lr=0.01)
classifier = main_3(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'sample-weight-simpler-lstmcnn.hdf5', epochs=50)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':


50
50
100
Train on 72696 samples, validate on 36736 samples
Epoch 1/50
72696/72696 [==============================] - 72s 997us/step - loss: 9.3860e-05 - acc: 0.9775 - val_loss: 0.0383 - val_acc: 0.9877
Epoch 2/50
72696/72696 [==============================] - 71s 976us/step - loss: 4.2672e-05 - acc: 0.9885 - val_loss: 0.0275 - val_acc: 0.9910
Epoch 3/50
72696/72696 [==============================] - 71s 972us/step - loss: 3.2742e-05 - acc: 0.9899 - val_loss: 0.0234 - val_acc: 0.9927
Epoch 4/50
72696/72696 [==============================] - 70s 963us/step - loss: 2.8194e-05 - acc: 0.9908 - val_loss: 0.0239 - val_acc: 0.9923
Epoch 5/50
72696/72696 [==============================] - 69s 952us/step - loss: 2.4658e-05 - acc: 0.9913 - val_loss: 0.0263 - val_acc: 0.9916
Epoch 6/50
72696/72696 [==============================] - 68s 936us/step - loss: 2.2302e-05 - acc: 0.9917 - val_loss: 0.0299 - val_acc: 0.9904
Epoch 7/50
72696/72696 [==============================] - 68s 936us/step - loss: 2

23088

In [54]:
classifier = main_3(loss='binary_crossentropy')
classifier.load_weights('sample-weight-simpler-lstmcnn.hdf5')
load_test_score(classifier, cnn_test, prevcnn_test, labels_test)

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':


(0.08483475, 0.9171657229965157)

In [16]:
# There is sample_weight
adam = keras.optimizers.Adam(lr=0.01)
classifier = main(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-model-1.hdf5', epochs=70)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


50
100
Train on 72696 samples, validate on 36736 samples
Epoch 1/70
72696/72696 [==============================] - 80s 1ms/step - loss: 2.0676e-04 - acc: 0.9509 - val_loss: 0.1713 - val_acc: 0.9456
Epoch 2/70
72696/72696 [==============================] - 75s 1ms/step - loss: 8.8679e-05 - acc: 0.9787 - val_loss: 0.0506 - val_acc: 0.9843
Epoch 3/70
72696/72696 [==============================] - 75s 1ms/step - loss: 7.0044e-05 - acc: 0.9827 - val_loss: 0.0386 - val_acc: 0.9884
Epoch 4/70
72696/72696 [==============================] - 75s 1ms/step - loss: 5.9352e-05 - acc: 0.9849 - val_loss: 0.0493 - val_acc: 0.9850
Epoch 5/70
72696/72696 [==============================] - 75s 1ms/step - loss: 5.2505e-05 - acc: 0.9861 - val_loss: 0.0414 - val_acc: 0.9876
Epoch 6/70
72696/72696 [==============================] - 75s 1ms/step - loss: 4.7743e-05 - acc: 0.9871 - val_loss: 0.0316 - val_acc: 0.9909
Epoch 7/70
72696/72696 [==============================] - 75s 1ms/step - loss: 4.4382e-05 - acc: 

53058

In [18]:
classifier = main(loss='binary_crossentropy')
classifier.load_weights('weight-model-1.hdf5')
load_test_score(classifier, cnn_test, prevcnn_test, labels_test)

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


(0.06571153, 0.9294425087108014)

In [19]:
# There is no sample_weight
adam = keras.optimizers.Adam(lr=0.01)
classifier = main(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-model-1-nosample.hdf5', epochs=100)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


50
100
Train on 72696 samples, validate on 36736 samples
Epoch 1/100
72696/72696 [==============================] - 78s 1ms/step - loss: 0.0751 - acc: 0.9740 - val_loss: 0.0955 - val_acc: 0.9757
Epoch 2/100
72696/72696 [==============================] - 77s 1ms/step - loss: 0.0333 - acc: 0.9899 - val_loss: 0.0539 - val_acc: 0.9851
Epoch 3/100
72696/72696 [==============================] - 75s 1ms/step - loss: 0.0274 - acc: 0.9918 - val_loss: 0.0306 - val_acc: 0.9912
Epoch 4/100
72696/72696 [==============================] - 75s 1ms/step - loss: 0.0237 - acc: 0.9931 - val_loss: 0.0307 - val_acc: 0.9917
Epoch 5/100
72696/72696 [==============================] - 77s 1ms/step - loss: 0.0216 - acc: 0.9937 - val_loss: 0.0196 - val_acc: 0.9947
Epoch 6/100
72696/72696 [==============================] - 78s 1ms/step - loss: 0.0199 - acc: 0.9944 - val_loss: 0.0183 - val_acc: 0.9951
Epoch 7/100
72696/72696 [==============================] - 79s 1ms/step - loss: 0.0186 - acc: 0.9947 - val_loss: 0.

39738

In [20]:
classifier = main(loss='binary_crossentropy')
classifier.load_weights('weight-model-1-nosample.hdf5')
load_test_score(classifier, cnn_test, prevcnn_test, labels_test)

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


(0.037136126, 0.9594131097560976)

In [21]:
# There is no sample_weight
adam = keras.optimizers.Adam(lr=0.01)
classifier = main_2(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-model-2-nosample.hdf5', epochs=100)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  import sys
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  # This is added back by InteractiveShellApp.init_path()
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


50
100
Train on 72696 samples, validate on 36736 samples
Epoch 1/100
72696/72696 [==============================] - 128s 2ms/step - loss: 0.0889 - acc: 0.9682 - val_loss: 0.0744 - val_acc: 0.9778
Epoch 2/100
72696/72696 [==============================] - 126s 2ms/step - loss: 0.0400 - acc: 0.9873 - val_loss: 0.0616 - val_acc: 0.9815
Epoch 3/100
72696/72696 [==============================] - 126s 2ms/step - loss: 0.0315 - acc: 0.9903 - val_loss: 0.0364 - val_acc: 0.9882
Epoch 4/100
72696/72696 [==============================] - 126s 2ms/step - loss: 0.0268 - acc: 0.9921 - val_loss: 0.4237 - val_acc: 0.9181
Epoch 5/100
72696/72696 [==============================] - 125s 2ms/step - loss: 0.0233 - acc: 0.9932 - val_loss: 1.2189 - val_acc: 0.8439
Epoch 6/100
72696/72696 [==============================] - 125s 2ms/step - loss: 0.0210 - acc: 0.9939 - val_loss: 0.0265 - val_acc: 0.9925
Epoch 7/100
72696/72696 [==============================] - 125s 2ms/step - loss: 0.0189 - acc: 0.9946 - val_l

KeyboardInterrupt: 